# Face Recognition Experiments

Using a variety of data sources, attempt to detect facial features and identify subjects.  We attemp to use pre-weighted VGG16 from Keras.

http://www.cl.cam.ac.uk/research/dtg/attarchive/facedatabase.html

http://www.face-rec.org/databases/


In [1]:
%matplotlib inline
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, GlobalAveragePooling2D, GlobalMaxPooling2D, Input, Flatten, Lambda, Dropout
from keras.models import Model, load_model
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras.applications import vgg16
from matplotlib import pyplot as plt
import numpy as np

Using Theano backend.


In [2]:
vgg_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32).reshape((3,1,1))
def vgg_preprocess(x):
    x = x - vgg_mean
    return x[:, ::-1] # reverse axis rgb->bgr

def create_default_model(nb_classes):
    # build the model from scratch using VGG16 as a base
    model_vgg16 = vgg16.VGG16(weights='imagenet', include_top=False)
    for layer in model_vgg16.layers:
        layer.trainable = False
    model_vgg16.summary()
    input_layer = Input(shape=(3,224,224),name = 'image_input')
    preprocess_layer = Lambda(vgg_preprocess, input_shape=(3,224,224), output_shape=(3,224,224))(input_layer)
    #Use the generated model 
    output_vgg16 = model_vgg16(preprocess_layer)

    #Add the fully-connected layers 
    x = Flatten(name='flatten')(output_vgg16)
    x = Dense(4096, activation='softmax', name='fc1')(x)
    x = Dropout(0.5)(x)
    x = Dense(4096, activation='relu', name='fc2')(x)
    x = Dropout(0.5)(x)
    x = Dense(nb_classes, activation='softmax', name='predictions')(x)
    model = Model(input=input_layer, output=x)
    return model

def add_compiler(model, lr=0.001):
    model.compile(optimizer=optimizers.Adam(lr=lr), loss='categorical_crossentropy', metrics=['accuracy'])
    
def load_model_with_weights_if_available(nb_classes, weights_file):
    # if weights exists on disk, then load it
    model = create_default_model(nb_classes)
    if os.path.exists(weights_file):
        model.load_weights(weights_file)
        print("Model loaded from file %s" % weights_file)
    else:
        print("Model built from scratch")
        
    add_compiler(model)
    model.summary()
    return model

def get_batches(path, gen=ImageDataGenerator(), shuffle=True, batch_size=8, class_mode='categorical'):
        return gen.flow_from_directory(path, target_size=(224,224),
                class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)

def plots(ims, figsize=(12,6), rows=1, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (len(ims.shape) == 4 and ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
    f = plt.figure(figsize=figsize)
    for i in range(len(ims)):
        sp = f.add_subplot(rows, len(ims)//rows, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i], interpolation=None if interp else 'none')

In [3]:
model = load_model_with_weights_if_available(2, "spectacles2.h5")

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 3, None, None) 0                                            
____________________________________________________________________________________________________
block1_conv1 (Convolution2D)     (None, 64, None, None)0           input_1[0][0]                    
____________________________________________________________________________________________________
block1_conv2 (Convolution2D)     (None, 64, None, None)0           block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_pool (MaxPooling2D)       (None, 64, None, None)0           block1_conv2[0][0]               
___________________________________________________________________________________________

In [6]:
batches = get_batches("yalefaces/features/spectacles", batch_size=5)
nb_epoch = 3
model.fit_generator(batches, batches.N, nb_epoch=nb_epoch)
model.save_weights("spectacles.h5")

Found 30 images belonging to 2 classes.
Epoch 1/3
30/30 [==============================] - 33s - loss: 0.6931 - acc: 0.5667     
Epoch 2/3
30/30 [==============================] - 32s - loss: 0.6857 - acc: 0.6000     
Epoc

In [11]:
model.evaluate_generator(batches, 10)

[0.68112009763717651, 0.60000000894069672]

In [7]:
faces_model = load_model_with_weights_if_available(40, "att_faces.h5")

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 3, None, None) 0                                            
____________________________________________________________________________________________________
block1_conv1 (Convolution2D)     (None, 64, None, None)0           input_2[0][0]                    
____________________________________________________________________________________________________
block1_conv2 (Convolution2D)     (None, 64, None, None)0           block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_pool (MaxPooling2D)       (None, 64, None, None)0           block1_conv2[0][0]               
___________________________________________________________________________________________

In [11]:
sample_batches = get_batches("att_faces_sample", batch_size=4, shuffle=True)
test_batches = get_batches("att_faces_test", batch_size=4, shuffle=True)
nb_epoch = 2
nb_val_samples=10
faces_model.fit_generator(sample_batches, sample_batches.N, nb_epoch, 
    validation_data=test_batches,
    nb_val_samples=nb_val_samples)

Found 120 images belonging to 40 classes.
Found 80 images belonging to 40 classes.
Epoch 1/2
120/120 [==============================] - 163s - loss: 3.7206 - acc: 0.0167 - val_loss: 3.7196 - val_acc: 0.0000e+00